In [223]:
import os
import time
import cv2
import requests
import json
import grpc
import numpy as np
import tensorflow as tf
from models.data_prepare import load_and_preprocess_image_pridict
from models.config import TABLE_PATH, TEST_PATH, OUTPUT_SHAPE

from concurrent import futures
from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2_grpc

## 配置信息

In [2]:
print (TABLE_PATH)

/root/python_project/crnn_by_tensorflow2.2.0/dataset/table.txt


In [3]:
#字典
with open(TABLE_PATH, 'r',encoding='utf8') as f:
    inv_table = [char.strip() for char in f]+[' ']*2

In [4]:
#测试数据集
test_all_image_paths = [TEST_PATH + img for img in sorted(os.listdir(TEST_PATH))]

## 调用tensorflow serving方法

In [190]:
def text_ocr(image_list):
    '''
    矩形框文字识别
    '''
    result_list = []
    input_image = []
    max_length = max([int(32*img.shape[1]/img.shape[0]) for img in image_list])
    for img in image_list:
        image = cv2.resize(img, (int(32*img.shape[1]/img.shape[0]), 32), 3)
        image = image.astype('float32')
        image /= 255
        image -= 0.5
        image /= 0.5
        padding_im = np.zeros((32, max_length, 3), dtype=np.float32)
        padding_im[:, 0:image.shape[1],:] = image
        input_image.append(padding_im.tolist())
    url = 'http://192.168.46.230:7501/v1/models/crnn:predict'
    headers = {"content-type": "application/json"}
    data = json.dumps({
            "signature_name": 'serving_default',
            "instances":input_image})
    start = time.time()
    predictions = requests.post(url,data=data, headers=headers,timeout=50).json()
    print('识别时间：', time.time()-start)
    if 'predictions' in predictions:
        if len(predictions['predictions']) > 0:
            for sentence in predictions['predictions']:
                result = ''
                for i in range(len(sentence)):
                    char_index = np.argmax(sentence[i])
                    if char_index != len(inv_table):
                        result +=  inv_table[char_index]
                result_list.append(result)
    return result_list

## gRPC调用方法

In [286]:
def text_ocr_grpc(image_list):
    '''
    矩形框文字识别
    '''
    result_list = []
    #grpc配置
    channel = grpc.insecure_channel('192.168.46.230:8500')
    stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)
    request = predict_pb2.PredictRequest()
    request.model_spec.name = 'crnn'
    request.model_spec.signature_name = 'serving_default'
    #循环遍历每个图片
    for img in image_list:
        image = cv2.resize(img, (int(32*img.shape[1]/img.shape[0]), 32), 3)
        image = image.astype('float32')
        image /= 255
        image -= 0.5
        image /= 0.5
        padding_im = np.expand_dims(image, axis=0)
        request.inputs['input_1'].CopyFrom(tf.make_tensor_proto(np.asarray(padding_im)))
        response = stub.Predict(request, 10.0)
        predictions = tf.make_ndarray(response.outputs["output_1"])
        for sentence in predictions:
            result = ''
            for i in range(len(sentence)):
                char_index = np.argmax(sentence[i])
                if char_index != len(inv_table):
                    result +=  inv_table[char_index]
            result_list.append(result)
    channel.close()
    return result_list

## 准备图片

In [287]:
image_list = []
for i in range(1,100):
    image_list.append(cv2.imread(test_all_image_paths[i]))

## 调用接口

In [288]:
start = time.time()
result = text_ocr_grpc(image_list)
print('识别时间：', time.time()-start)
print('识别结果：', result)

识别时间： 4.706458568572998
识别结果： ['血脂4项(LIP4)(番禺)', '1.16', '1.19', '(PY.Anti-HBe.)', '阴性:>-！阳性:<1', 'PY.乙肝病病毒核心抗体定量', '0.010', '(PY.Anti-HBe.)', ' 结   肝病毒核心抗体定量检查阳性乙肝病毒表 抗体定量检查阳性', '性别:男', '年龄 38', '第5页/共9页', '1..42', '低密度脂蛋白胆固醇(LDL-C)', '2.86', 'mmol//L', '1.50', '3.37', ' 结  油三酯 TG))升  PY 总胆固醇(PY.CHOL))升 ', '前列腺抗原两项(番禺))', '检查日期:2019-12-30', '检查 期 2019-12-28', '项目名称', '检查结果', '参考考值', 'pY.总前列腺特异性抗原', '1.56', 'ng/ml', '(PY..TPSAA', '<4', 'pY游离前列腺特异性抗原', '0..523', '项目名 ', 'ng/ml', '((PY.FPSA)', 'PY 游离PSA//总', '0.34', 'PSA(PY. PSA-Ratio', '>25%BPH风险高', '<<25%前列腺CA风', '险高', '小结:未见明显显异常', '肾功5项(S5)(番禺)', '检查医生 杨玉娟', '检查 期:2019-12-28', '检查医生:杨玉娟', '项目名称', '检查结果', '单位', '提示', '参考值', 'PY.尿酸(PY.UA)', '556', 'umol//1', '检查结 ', '升高', '90', '420', '尿素.(PY.UUREA))', '4.2', 'mmmol/1', '2.86', '8.2', '肌酐.(PY.CREA)', '900.00', '高密度脂蛋白胆固醇(HDDL--C)', 'umol//1', '62', '1115', '白蛋白(ALB)', '41.0', 'g L', '胱抑素素C(CysC)', '0.85', 'mg/L', '0..59', '参考值', '1..03', '小结:PY尿酸((PPY.UA))升高', '乙肝两对半定量((番禺)', '检查 期 201